In [6]:
import torch

# Embedding - czyli zamienianie słów na wektory

In [7]:
sentence = 'Life is short, eat dessert first'

dc = {s:i for i,s in enumerate(sorted(sentence.replace(',', '').split()))}
print(dc)

{'Life': 0, 'dessert': 1, 'eat': 2, 'first': 3, 'is': 4, 'short': 5}


In [8]:
sentence_int = torch.tensor([dc[s] for s in sentence.replace(',', '').split()])
print(sentence_int)

tensor([0, 4, 5, 2, 1, 3])


In [9]:
torch.manual_seed(123)
embed = torch.nn.Embedding(6, 16)
embedded_sentence = embed(sentence_int).detach()

In [10]:
print(embedded_sentence)
print(embedded_sentence.shape)

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
          0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692],
        [ 0.5146,  0.9938, -0.2587, -1.0826, -0.0444,  1.6236, -2.3229,  1.0878,
          0.6716,  0.6933, -0.9487, -0.0765, -0.1526,  0.1167,  0.4403, -1.4465],
        [ 0.2553, -0.5496,  1.0042,  0.8272, -0.3948,  0.4892, -0.2168, -1.7472,
         -1.6025, -1.0764,  0.9031, -0.7218, -0.5951, -0.7112,  0.6230, -1.3729],
        [-1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343, -0.4991, -1.0867,
          0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [-0.0770, -1.0205, -0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010,
          0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255, -0.6315, -2.8400],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
          2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,  0.2293]])
torch.Size([6, 16])


# Macierze wag

Teraz omówmy szeroko stosowany mechanizm samo-uwagi znany jako **scaled dot-product attention**, który stanowi integralny element architektury transformera.

Mechanizm samo-uwagi wykorzystuje trzy macierze wag: **W<sub>q</sub>**, **W<sub>k</sub>** oraz **W<sub>v</sub>**, które są dostrajane jako parametry modelu podczas uczenia.  
Macierze te służą do rzutowania wektorów wejściowych na odpowiednie reprezentacje: **zapytania (query)**, **klucze (key)** i **wartości (value)**.

Odpowiednie sekwencje zapytań, kluczy i wartości otrzymuje się przez mnożenie macierzy wag **W** przez wektory osadzeń wejściowych **x**:

- **Sekwencja zapytań:**  
  $$
  \mathbf{q}^{(i)} = \mathbf{W}_q \mathbf{x}^{(i)} \quad \text{dla } i \in [1, T]
  $$

- **Sekwencja kluczy:**  
  $$
  \mathbf{k}^{(i)} = \mathbf{W}_k \mathbf{x}^{(i)} \quad \text{dla } i \in [1, T]
  $$

- **Sekwencja wartości:**  
  $$
  \mathbf{v}^{(i)} = \mathbf{W}_v \mathbf{x}^{(i)} \quad \text{dla } i \in [1, T]
  $$

Indeks *i* odnosi się do pozycji tokena w sekwencji wejściowej, której długość wynosi *T*.


Zarówno **q<sup>(i)</sup>**, jak i **k<sup>(i)</sup>** są wektorami o wymiarze *dₖ*. Macierze projekcji **W<sub>q</sub>** oraz **W<sub>k</sub>** mają rozmiar $d_k \times d$, natomiast macierz **W<sub>v</sub>** ma rozmiar $d_v \times d$. Warto zauważyć, że *d* reprezentuje rozmiar każdego wektora słowa *x*. Ponieważ podczas obliczania iloczynu skalarnego między wektorami zapytań i kluczy wymagane jest, aby te dwa wektory miały tę samą liczbę elementów ($d_q = d_k$), liczba elementów w wektorze wartości **v<sup>(i)</sup>**, która determinuje rozmiar wynikowego wektora kontekstu, może być dowolna. W dalszej części przykładu kodu przyjmujemy więc, że $d_q = d_k = 24$ oraz $d_v = 28$, inicjalizując macierze projekcji w następujący sposób.


In [11]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]

d_q, d_k, d_v = 24, 24, 28

W_query = torch.nn.Parameter(torch.rand(d_q, d))
W_key = torch.nn.Parameter(torch.rand(d_k, d))
W_value = torch.nn.Parameter(torch.rand(d_v, d))


In [16]:
print("W_query:", W_query.shape)
print("W_key:  ", W_key.shape)
print("W_value:", W_value.shape)

W_query: torch.Size([24, 16])
W_key:   torch.Size([24, 16])
W_value: torch.Size([28, 16])


In [17]:
x_2 = embedded_sentence[1]
query_2 = W_query.matmul(x_2)
key_2 = W_key.matmul(x_2)
value_2 = W_value.matmul(x_2)

print(query_2.shape)
print(key_2.shape)
print(value_2.shape)


torch.Size([24])
torch.Size([24])
torch.Size([28])


In [19]:
keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)


keys.shape: torch.Size([6, 24])
values.shape: torch.Size([6, 28])


Jak pokazano na ilustracji powyżej, obliczamy $ \omega_{i,j} $ jako iloczyn skalarny pomiędzy sekwencjami zapytań i kluczy:

$$ \omega_{i,j} = \mathbf{q}^{(i)^\top} \mathbf{k}^{(j)} $$

Na przykład możemy obliczyć nieznormalizowaną wagę atencji (attention weight) dla zapytania i piątego elementu wejściowego (odpowiadającego pozycji indeksu 4) w następujący sposób:


In [20]:
omega_24 = query_2.dot(keys[4])
print(omega_24)

tensor(11.1466, grad_fn=<DotBackward0>)


Mówi to nam jak bardzo token na pozycji 2 (czyli "is") jest powiązany z tokenem na pozycji 4 (czyli "dessert").

Ponieważ będziemy potrzebować tych wartości do obliczenia wag atencji w dalszej części, obliczmy wartości $ \omega $ dla wszystkich tokenów wejściowych, tak jak pokazano na poprzedniej ilustracji:


In [21]:
omega_2 = query_2.matmul(keys.T)
print(omega_2)

tensor([ 8.5808, -7.6597,  3.2558,  1.0395, 11.1466, -0.4800],
       grad_fn=<SqueezeBackward4>)


Skalowanie przez $d_k$ zapewnia, że długość euklidesowa wektorów wag pozostaje w przybliżeniu na tym samym poziomie. Pomaga to zapobiec sytuacji, w której wagi uwagi stają się zbyt małe lub zbyt duże, co mogłoby prowadzić do niestabilności numerycznej lub utrudniać zbieżność modelu podczas uczenia.

W kodzie obliczanie wag uwagi można zaimplementować w następujący sposób:


In [24]:
import torch.nn.functional as F

attention_weights_2 = F.softmax(omega_2 / d_k**0.5, dim=0)
print(attention_weights_2)


tensor([0.2912, 0.0106, 0.0982, 0.0625, 0.4917, 0.0458],
       grad_fn=<SoftmaxBackward0>)


Ostatecznym krokiem jest obliczenie wektora kontekstu $ \mathbf{z}^{(2)} $, który stanowi wersję naszego oryginalnego wektora zapytania $ \mathbf{x}^{(2)} $, ważoną przez wagi uwagi. Wektor ten uwzględnia wszystkie pozostałe elementy wejściowe jako kontekst poprzez wagi uwagi:


In [25]:
context_vector_2 = attention_weights_2.matmul(values)

print(context_vector_2.shape)
print(context_vector_2)

torch.Size([28])
tensor([-1.5993,  0.0156,  1.2670,  0.0032, -0.6460, -1.1407, -0.4908, -1.4632,
         0.4747,  1.1926,  0.4506, -0.7110,  0.0602,  0.7125, -0.1628, -2.0184,
         0.3838, -2.1188, -0.8136, -1.5694,  0.7934, -0.2911, -1.3640, -0.2366,
        -0.9564, -0.5265,  0.0624,  1.7084], grad_fn=<SqueezeBackward4>)


Zauważ, że ten wektor wyjściowy ma więcej wymiarów ($d_v = 28$) niż oryginalny wektor wejściowy ($d = 16$), ponieważ wcześniej przyjęliśmy $d_v > d$. Wybór rozmiaru osadzenia (embeddingu) jest jednak arbitralny.

### 🧭 Podsumowanie kroków obliczania samo-uwagi (scaled dot-product attention)

<p align="center">
  <img src="context-vector.png" alt="Wektor kontekstu" width="600"/>
  <img src="single-head.png" alt="Macierze QKV" width="600"/>
</p>

1. **Embedding** – wejściowe słowa (tokeny) zostały zamienione na wektory o ustalonym rozmiarze $d$.  
   Każdy token w sekwencji ma więc reprezentację numeryczną $\mathbf{x}^{(i)} \in \mathbb{R}^d$.

2. **Projekcje na zapytania, klucze i wartości** – wektory osadzeń $\mathbf{x}^{(i)}$ zostały przekształcone przy pomocy trzech macierzy wag:  
   $$
   \mathbf{q}^{(i)} = \mathbf{W}_q \mathbf{x}^{(i)}, \quad
   \mathbf{k}^{(i)} = \mathbf{W}_k \mathbf{x}^{(i)}, \quad
   \mathbf{v}^{(i)} = \mathbf{W}_v \mathbf{x}^{(i)}
   $$
   gdzie $\mathbf{W}_q$, $\mathbf{W}_k$, $\mathbf{W}_v$ mają odpowiednio wymiary $(d_q \times d)$, $(d_k \times d)$ i $(d_v \times d)$.

3. **Obliczenie surowych wag uwagi** – dla każdej pary tokenów $(i, j)$ obliczono współczynnik podobieństwa (ang. *attention score*) jako iloczyn skalarny:  
   $$
   \omega_{i,j} = \mathbf{q}^{(i)^\top} \mathbf{k}^{(j)}
   $$

4. **Skalowanie przez $d_k$** – wartości $\omega_{i,j}$ zostały przeskalowane przez $\sqrt{d_k}$, aby utrzymać stabilną wielkość gradientów i zapobiec zbyt dużym lub małym wartościom uwagi.

5. **Normalizacja (softmax)** – przeskalowane wagi zostały znormalizowane funkcją softmax, tak aby ich suma dla każdego zapytania wynosiła 1 (interpretacja jako rozkład prawdopodobieństwa).

6. **Obliczenie wektora kontekstu** – dla każdego tokena $i$ obliczono wektor kontekstu jako średnią ważoną wektorów wartości $\mathbf{v}^{(j)}$ z wagami uwagi:  
   $$
   \mathbf{z}^{(i)} = \sum_j \text{softmax}(\omega_{i,j}) \, \mathbf{v}^{(j)}
   $$

7. **Interpretacja** – otrzymany wektor $\mathbf{z}^{(i)}$ jest nową reprezentacją tokena, która uwzględnia jego kontekst w całej sekwencji (czyli „na co token zwraca uwagę”).

nasze przykładowe zdanie:  
**"Life is short, eat dessert first"**

Słownik tokenów:  
`{'Life': 0, 'dessert': 1, 'eat': 2, 'first': 3, 'is': 4, 'short': 5}`

| Symbol | Znaczenie | Typ danych | Rola | Przykład (dla zdania *Life is short, eat dessert first*) |
|:--|:--|:--|:--|:--|
| **$\mathbf{x}^{(i)}$** | wektor osadzenia (*embedding*) tokena wejściowego | wektor $\in \mathbb{R}^d$ | „Jak wygląda moje słowo w przestrzeni cech?” | wektor reprezentujący znaczenie słowa **Life** w przestrzeni embeddingów |
| **$\mathbf{q}^{(i)}$** | wektor zapytania (*query*) | wektor $\in \mathbb{R}^{d_q}$ | „Czego szukam?” | wektor określający, czego **Life** szuka w innych słowach zdania |
| **$\mathbf{k}^{(j)}$** | wektor klucza (*key*) | wektor $\in \mathbb{R}^{d_k}$ | „Jakie mam cechy?” | wektor opisujący cechy słowa **dessert**, które mogą przyciągnąć uwagę |
| **$\mathbf{v}^{(j)}$** | wektor wartości (*value*) | wektor $\in \mathbb{R}^{d_v}$ | „Jaką niosę informację?” | reprezentacja semantyczna słowa **dessert**, która może być przekazana dalej |
| **$\omega_{i,j}$** | surowy wynik podobieństwa | skalar | „Na ile podobni jesteśmy?” | wartość określająca podobieństwo między **Life** a **dessert** |
| **$\alpha_{i,j}$** | waga uwagi (po softmaxie) | skalar | „Jak bardzo się liczę?” | waga określająca, ile uwagi **Life** poświęca słowu **dessert** |
| **$\mathbf{z}^{(i)}$** | wektor kontekstu | wektor $\in \mathbb{R}^{d_v}$ | „Nowa reprezentacja tokena *i*” | nowa reprezentacja **Life**, uwzględniająca kontekst całego zdania |


# Multi-Head Attention

Jak sama nazwa wskazuje, **multi-head attention** (uwaga wielogłowa) obejmuje wiele takich głów — każda z nich składa się z własnych macierzy **zapytania (query)**, **klucza (key)** oraz **wartości (value)**.  
Koncepcja ta jest analogiczna do wykorzystania wielu jąder (*kernels*) w konwolucyjnych sieciach neuronowych (CNN).

Aby zilustrować to w kodzie, załóżmy, że mamy **3 głowy uwagi**.  
W takim przypadku rozszerzamy macierze wag z wymiaru $d' \times d$ do $3 \times d' \times d$, tak aby każda głowa miała swój własny zestaw wag:

In [30]:
h = 3
multihead_W_query = torch.nn.Parameter(torch.rand(h, d_q, d))
multihead_W_key = torch.nn.Parameter(torch.rand(h, d_k, d))
multihead_W_value = torch.nn.Parameter(torch.rand(h, d_v, d))


In [31]:
multihead_query_2 = multihead_W_query.matmul(x_2)
print(multihead_query_2.shape)

torch.Size([3, 24])


In [32]:
multihead_key_2 = multihead_W_key.matmul(x_2)
multihead_value_2 = multihead_W_value.matmul(x_2)

In [33]:
stacked_inputs = embedded_sentence.T.repeat(3, 1, 1)
print(stacked_inputs.shape)


torch.Size([3, 16, 6])


In [34]:
multihead_keys = torch.bmm(multihead_W_key, stacked_inputs)
multihead_values = torch.bmm(multihead_W_value, stacked_inputs)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)

multihead_keys.shape: torch.Size([3, 24, 6])
multihead_values.shape: torch.Size([3, 28, 6])


In [35]:
multihead_keys = multihead_keys.permute(0, 2, 1)
multihead_values = multihead_values.permute(0, 2, 1)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)


multihead_keys.shape: torch.Size([3, 6, 24])
multihead_values.shape: torch.Size([3, 6, 28])
